# QC-Py-07 : Futures et Forex Trading dans QuantConnect

**Duree estimee** : 75 minutes

**Objectifs d'apprentissage** :
1. Comprendre les contrats Futures et leur fonctionnement (marge, rollover)
2. Maitriser les Futures Chains et la selection de contrats
3. Trader des paires Forex (EURUSD, GBPUSD, USDJPY)
4. Gerer le leverage et le position sizing de maniere responsable
5. Implementer une strategie Trend Following complete sur ES (S&P 500 E-mini)

**Prerequis** :
- Notebooks QC-Py-01 a QC-Py-04 completes
- Comprehension du lifecycle QCAlgorithm
- Notions de base sur les marches derives

---

## Structure du Notebook

| Partie | Contenu | Duree |
|--------|---------|-------|
| **1** | Introduction aux Futures | 10 min |
| **2** | Ajouter des Futures | 10 min |
| **3** | Futures Chain et Contract Selection | 10 min |
| **4** | Rollover (Roulement) | 10 min |
| **5** | Introduction au Forex | 10 min |
| **6** | Ajouter des Forex | 10 min |
| **7** | Trading Forex | 15 min |
| **8** | Gestion du Leverage | 10 min |
| **9** | Position Sizing avec Leverage | 10 min |
| **10** | Strategie Complete Trend Following | 20 min |

---

# PARTIE 1 : FUTURES (40 min)

---

## 1. Introduction aux Futures (10 min)

### Qu'est-ce qu'un contrat Future ?

Un **contrat Future** (ou contrat a terme) est un accord standardise pour acheter ou vendre un actif sous-jacent a un prix determine, a une date d'expiration future.

**Caracteristiques cles** :
- **Standardisation** : Quantite, qualite, date d'expiration fixes
- **Effet de levier** : Seule une fraction de la valeur (marge) est requise
- **Expiration** : Les contrats expirent a des dates specifiques
- **Settlement** : Physique (livraison) ou cash (difference de prix)

### Marge vs Cash Trading

| Aspect | Cash Trading (Actions) | Futures (Margin) |
|--------|------------------------|------------------|
| **Capital requis** | 100% de la valeur | 5-15% (marge initiale) |
| **Leverage** | 1:1 (ou 2:1 avec margin account) | 10:1 a 20:1 typique |
| **Mark-to-Market** | Non | Oui (MTM quotidien) |
| **Risque** | Perte limitee a l'investissement | Perte potentiellement illimitee |
| **Appel de marge** | Rare | Frequent si position perdante |

### Types de Marge

- **Marge Initiale** (Initial Margin) : Depot requis pour ouvrir une position
- **Marge de Maintenance** (Maintenance Margin) : Seuil minimum a maintenir
- **Appel de Marge** (Margin Call) : Si capital < marge maintenance, depot additionnel requis

### Principaux Marches Futures

| Symbole | Nom | Sous-jacent | Multiplicateur | Exchange |
|---------|-----|-------------|----------------|----------|
| **ES** | E-mini S&P 500 | Indice S&P 500 | $50 | CME |
| **NQ** | E-mini Nasdaq 100 | Indice Nasdaq 100 | $20 | CME |
| **CL** | Crude Oil | Petrole brut WTI | $1,000 | NYMEX |
| **GC** | Gold | Or | $100 | COMEX |
| **ZB** | 30-Year T-Bond | Obligation US 30 ans | $1,000 | CBOT |
| **ZC** | Corn | Mais | $50 | CBOT |

### Calcul de la Valeur d'un Contrat

**Formule** : `Valeur du Contrat = Prix du Future x Multiplicateur`

**Exemple ES** :
- Prix ES : 5,000 points
- Multiplicateur : $50
- Valeur : 5,000 x $50 = **$250,000** par contrat

**Marge typique ES** (~5%) : ~$12,500 pour controler $250,000

In [ ]:
# Demonstration calcul valeur contrat et marge
# Ce code peut etre execute localement pour comprendre les calculs

# Parametres ES E-mini S&P 500
es_price = 5000  # Prix en points
es_multiplier = 50  # $50 par point

# Calcul valeur du contrat
contract_value = es_price * es_multiplier
print(f"=== E-mini S&P 500 (ES) ===")
print(f"Prix: {es_price:,} points")
print(f"Multiplicateur: ${es_multiplier}")
print(f"Valeur du contrat: ${contract_value:,}")

# Marges typiques
initial_margin_pct = 0.05  # 5%
maintenance_margin_pct = 0.04  # 4%

initial_margin = contract_value * initial_margin_pct
maintenance_margin = contract_value * maintenance_margin_pct

print(f"\nMarges:")
print(f"  Marge Initiale (~5%): ${initial_margin:,.0f}")
print(f"  Marge Maintenance (~4%): ${maintenance_margin:,.0f}")

# Leverage effectif
leverage = contract_value / initial_margin
print(f"\nLeverage effectif: {leverage:.0f}:1")

In [ ]:
# Comparaison plusieurs contrats Futures

futures_contracts = [
    {"symbol": "ES", "name": "E-mini S&P 500", "price": 5000, "multiplier": 50},
    {"symbol": "NQ", "name": "E-mini Nasdaq 100", "price": 18000, "multiplier": 20},
    {"symbol": "CL", "name": "Crude Oil", "price": 75, "multiplier": 1000},
    {"symbol": "GC", "name": "Gold", "price": 2000, "multiplier": 100},
    {"symbol": "ZB", "name": "30-Year T-Bond", "price": 120, "multiplier": 1000},
]

print(f"{'Symbol':<8} {'Name':<20} {'Price':>10} {'Mult':>8} {'Contract Value':>15}")
print("-" * 65)

for f in futures_contracts:
    value = f["price"] * f["multiplier"]
    print(f"{f['symbol']:<8} {f['name']:<20} {f['price']:>10,} {f['multiplier']:>8} ${value:>14,}")

### Cycle d'Expiration des Futures

Les contrats Futures expirent selon des cycles specifiques :

| Cycle | Mois | Symboles |
|-------|------|----------|
| **Trimestriel** | Mars, Juin, Sept, Dec | H, M, U, Z |
| **Mensuel** | Tous les mois | F, G, H, J, K, M, N, Q, U, V, X, Z |

**Notation standard** : `ESH24` = ES (S&P 500), Mars (H), 2024

**Symboles des mois** :
```
F = Janvier    G = Fevrier    H = Mars
J = Avril      K = Mai        M = Juin
N = Juillet    Q = Aout       U = Septembre
V = Octobre    X = Novembre   Z = Decembre
```

---

## 2. Ajouter des Futures dans QuantConnect (10 min)

### Deux Approches : Continuous vs Specific

QuantConnect propose deux manieres d'ajouter des Futures :

| Approche | Description | Usage |
|----------|-------------|-------|
| **Continuous Contract** | QuantConnect gere automatiquement le rollover | Recommande pour la plupart des strategies |
| **Specific Contract** | Vous specifiez exactement quel contrat trader | Calendrier spreads, strategies specifiques |

### Methode 1 : Continuous Contract (Recommande)

Le **Continuous Contract** est une abstraction qui suit automatiquement le contrat le plus liquide et gere le rollover.

In [ ]:
# Code QuantConnect - Ajouter un Future avec Continuous Contract
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class FuturesContinuousExample(QCAlgorithm):
    """
    Exemple d'ajout de Futures avec Continuous Contract.
    Le rollover est gere automatiquement.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter ES (E-mini S&P 500) comme Continuous Contract
        self.es = self.AddFuture(
            Futures.Indices.SP500EMini,  # Utiliser l'enum Futures
            Resolution.Minute
        )
        
        # Filter: Contrats expirant dans les 90 prochains jours
        self.es.SetFilter(timedelta(0), timedelta(90))
        
        self.Debug(f"ES Future ajoute: {self.es.Symbol}")
    
    def OnData(self, data):
        pass  # Trading logic here

### SetFilter : Controler les Contrats Recus

La methode `SetFilter()` controle quels contrats sont inclus dans la FutureChain :

```python
# Syntaxe: SetFilter(minExpiry, maxExpiry)
self.es.SetFilter(timedelta(0), timedelta(90))  # 0 a 90 jours

# Ou avec jours specifiques
self.es.SetFilter(0, 90)  # Equivalent (0 a 90 jours)

# Filtrage avance avec lambda
self.es.SetFilter(lambda universe: universe.Expiration(0, 90).OnlyApplyFilterAtMarketOpen())
```

**Recommandation** : Utiliser `timedelta(0), timedelta(90)` pour avoir le front-month et quelques contrats suivants.

### Methode 2 : Specific Contract (Avance)

Pour trader un contrat specifique (ex: strategies de calendar spread) :

In [ ]:
# Code QuantConnect - Ajouter un contrat Future specifique
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class FuturesSpecificContractExample(QCAlgorithm):
    """
    Exemple d'ajout d'un contrat Future specifique.
    Utile pour calendar spreads ou strategies specifiques.
    """
    
    def Initialize(self):
        self.SetStartDate(2024, 1, 1)
        self.SetEndDate(2024, 3, 15)  # Avant expiration Mars
        self.SetCash(100000)
        
        # Ajouter un contrat specifique: ES Mars 2024
        # Format: CreateFuture(ticker, market, expiry)
        es_march_symbol = Symbol.CreateFuture(
            "ES",
            Market.CME,
            datetime(2024, 3, 15)  # Date d'expiration
        )
        
        self.es_march = self.AddFutureContract(es_march_symbol, Resolution.Minute)
        
        self.Debug(f"ES Mars 2024 ajoute: {self.es_march.Symbol}")
    
    def OnData(self, data):
        pass

### Principaux Symboles Futures QuantConnect

QuantConnect fournit un enum `Futures` pour acceder facilement aux symboles :

```python
# Indices
Futures.Indices.SP500EMini      # ES
Futures.Indices.NASDAQ100EMini  # NQ
Futures.Indices.Dow30EMini      # YM

# Energie
Futures.Energies.CrudeOilWTI    # CL
Futures.Energies.NaturalGas     # NG

# Metaux
Futures.Metals.Gold             # GC
Futures.Metals.Silver           # SI

# Taux d'interet
Futures.Financials.Y30TreasuryBond  # ZB
Futures.Financials.Y10TreasuryNote  # ZN

# Agriculture
Futures.Grains.Corn             # ZC
Futures.Grains.Wheat            # ZW
Futures.Grains.Soybeans         # ZS
```

---

## 3. Futures Chain et Contract Selection (10 min)

### Comprendre la Futures Chain

Quand vous ajoutez un Future avec `AddFuture()`, QuantConnect vous fournit une **FutureChain** contenant tous les contrats disponibles selon votre filtre.

```python
# Dans OnData(self, slice):
for chain in slice.FutureChains.Values:
    # chain contient tous les contrats disponibles
    for contract in chain:
        # contract.Symbol, contract.Expiry, contract.OpenInterest, etc.
        pass
```

### Proprietes d'un FutureContract

| Propriete | Description |
|-----------|-------------|
| `Symbol` | Symbole unique du contrat |
| `Expiry` | Date d'expiration |
| `OpenInterest` | Nombre de contrats ouverts |
| `LastPrice` | Dernier prix negocie |
| `BidPrice` / `AskPrice` | Prix bid/ask |
| `Volume` | Volume de trading |

### Selection du Front-Month Contract

Le **front-month** est le contrat avec l'expiration la plus proche (generalement le plus liquide).

In [ ]:
# Code QuantConnect - Selection du Front-Month Contract
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class FrontMonthSelection(QCAlgorithm):
    """
    Strategie qui trade toujours le contrat front-month.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter ES
        self.es = self.AddFuture(Futures.Indices.SP500EMini, Resolution.Minute)
        self.es.SetFilter(timedelta(0), timedelta(90))
        
        # Variable pour stocker le contrat actif
        self.current_contract = None
    
    def OnData(self, slice):
        # Iterer sur les FutureChains disponibles
        for chain in slice.FutureChains.Values:
            # Verifier que la chain n'est pas vide
            if len(list(chain)) == 0:
                continue
            
            # Selection du front-month (expiration la plus proche)
            front_contract = min(chain, key=lambda x: x.Expiry)
            
            # Afficher informations
            self.Debug(f"{self.Time}: Front-month: {front_contract.Symbol}")
            self.Debug(f"  Expiry: {front_contract.Expiry}")
            self.Debug(f"  Last Price: {front_contract.LastPrice}")
            self.Debug(f"  Open Interest: {front_contract.OpenInterest}")
            
            # Mettre a jour le contrat actif
            self.current_contract = front_contract.Symbol

### Selection par Open Interest (Plus Liquide)

In [ ]:
# Code QuantConnect - Selection par Open Interest
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class MostLiquidSelection(QCAlgorithm):
    """
    Strategie qui trade le contrat le plus liquide (highest open interest).
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.es = self.AddFuture(Futures.Indices.SP500EMini, Resolution.Minute)
        self.es.SetFilter(timedelta(0), timedelta(90))
        
        self.current_contract = None
    
    def OnData(self, slice):
        for chain in slice.FutureChains.Values:
            contracts = list(chain)
            if len(contracts) == 0:
                continue
            
            # Selection par open interest (le plus liquide)
            most_liquid = max(contracts, key=lambda x: x.OpenInterest)
            
            self.Debug(f"{self.Time}: Most Liquid: {most_liquid.Symbol}")
            self.Debug(f"  Expiry: {most_liquid.Expiry}")
            self.Debug(f"  Open Interest: {most_liquid.OpenInterest}")
            
            self.current_contract = most_liquid.Symbol

### Comparaison des Methodes de Selection

| Methode | Avantages | Inconvenients |
|---------|-----------|---------------|
| **Front-Month** | Simple, previsible | Peut etre moins liquide proche expiration |
| **Max Open Interest** | Plus liquide | Peut changer avant expiration |
| **Max Volume** | Activite recente | Plus volatile |

> **Recommandation** : Pour la plupart des strategies, utilisez le **front-month** et gerez le rollover avant expiration.

---

## 4. Rollover (Roulement) (10 min)

### Le Probleme : Les Futures Expirent

Contrairement aux actions, les contrats Futures ont une **date d'expiration**. Si vous gardez une position jusqu'a expiration :

- **Settlement cash** : Vous recevez/payez la difference en cash
- **Settlement physique** : Vous devez prendre/livrer l'actif sous-jacent (rare pour particuliers)

### La Solution : Le Rollover

Le **rollover** consiste a fermer la position sur le contrat expirant et ouvrir une position equivalente sur le contrat suivant.

```
Avant rollover:
  Position: Long 2 ES Mars 2024
  
Rollover (ex: 5 jours avant expiration):
  1. Vendre 2 ES Mars 2024
  2. Acheter 2 ES Juin 2024
  
Apres rollover:
  Position: Long 2 ES Juin 2024
```

### Detection du Nouveau Contrat avec OnSecuritiesChanged

In [ ]:
# Code QuantConnect - Rollover Automatique avec OnSecuritiesChanged
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class FuturesRolloverExample(QCAlgorithm):
    """
    Strategie avec gestion du rollover automatique.
    Utilise OnSecuritiesChanged pour detecter les nouveaux contrats.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter ES
        self.es = self.AddFuture(
            Futures.Indices.SP500EMini,
            Resolution.Minute
        )
        self.es.SetFilter(timedelta(0), timedelta(90))
        
        # Tracking du contrat actif
        self.current_contract = None
        self.rollover_days_before_expiry = 5  # Rollover 5 jours avant expiration
    
    def OnSecuritiesChanged(self, changes):
        """
        Appele quand des securities sont ajoutees ou retirees.
        Utilise pour detecter le rollover de contrat.
        """
        for security in changes.AddedSecurities:
            if security.Type == SecurityType.Future:
                # Nouveau contrat disponible
                self.Debug(f"{self.Time}: Nouveau contrat detecte: {security.Symbol}")
                self.Debug(f"  Expiry: {security.Symbol.ID.Date}")
        
        for security in changes.RemovedSecurities:
            if security.Type == SecurityType.Future:
                # Contrat retire (expire ou hors filtre)
                self.Debug(f"{self.Time}: Contrat retire: {security.Symbol}")
    
    def OnData(self, slice):
        for chain in slice.FutureChains.Values:
            contracts = list(chain)
            if len(contracts) == 0:
                continue
            
            # Selectionner le front-month
            front_contract = min(contracts, key=lambda x: x.Expiry)
            
            # Verifier si rollover necessaire
            days_to_expiry = (front_contract.Expiry - self.Time).days
            
            if days_to_expiry <= self.rollover_days_before_expiry:
                # Trouver le prochain contrat
                next_contracts = [c for c in contracts if c.Expiry > front_contract.Expiry]
                if len(next_contracts) > 0:
                    next_contract = min(next_contracts, key=lambda x: x.Expiry)
                    self._perform_rollover(front_contract, next_contract)
            else:
                self.current_contract = front_contract.Symbol
    
    def _perform_rollover(self, old_contract, new_contract):
        """
        Execute le rollover d'un contrat vers un autre.
        """
        self.Debug(f"{self.Time}: ROLLOVER")
        self.Debug(f"  De: {old_contract.Symbol} (expiry: {old_contract.Expiry})")
        self.Debug(f"  Vers: {new_contract.Symbol} (expiry: {new_contract.Expiry})")
        
        # Si position ouverte sur l'ancien contrat, la transferer
        if self.Portfolio[old_contract.Symbol].Invested:
            # Recuperer la quantite actuelle
            qty = self.Portfolio[old_contract.Symbol].Quantity
            
            # Fermer l'ancienne position
            self.Liquidate(old_contract.Symbol)
            
            # Ouvrir sur le nouveau contrat
            self.MarketOrder(new_contract.Symbol, qty)
            
            self.Debug(f"  Position transferee: {qty} contrats")
        
        # Mettre a jour le contrat actif
        self.current_contract = new_contract.Symbol

### Timing du Rollover

**Quand rouler ?**

| Approche | Description | Utilisation |
|----------|-------------|-------------|
| **Jours avant expiration** | Roll X jours avant (ex: 5 jours) | Simple, previsible |
| **Open Interest** | Roll quand OI du nouveau > OI de l'ancien | Suit la liquidite |
| **Volume** | Roll quand volume migre | Reactif |
| **Calendar date** | Roll a une date fixe du mois | Institutionnel |

**Couts du Rollover** :
- Spread bid/ask x 2 (fermeture + ouverture)
- Slippage potentiel
- Contango/Backwardation (difference de prix entre contrats)

> **Contango** : Nouveau contrat plus cher que l'ancien (cout de portage positif)  
> **Backwardation** : Nouveau contrat moins cher (benefice au rollover)

---

# PARTIE 2 : FOREX (35 min)

---

## 5. Introduction au Forex (10 min)

### Qu'est-ce que le Forex ?

Le **Forex** (Foreign Exchange) est le marche des changes, ou s'echangent les devises mondiales.

**Caracteristiques** :
- **Plus grand marche mondial** : ~$6 trillions/jour de volume
- **24h/24, 5j/7** : Ouvert du dimanche soir au vendredi soir (GMT)
- **Decentralise** : Pas de bourse centrale (OTC)
- **Highly leveraged** : Leverage 50:1 a 500:1 disponible

### Paires de Devises

Les devises sont toujours tradees en **paires** :

| Paire | Nom | Base | Quote |
|-------|-----|------|-------|
| **EURUSD** | Euro/Dollar | EUR | USD |
| **GBPUSD** | Cable | GBP | USD |
| **USDJPY** | Dollar/Yen | USD | JPY |
| **AUDUSD** | Aussie | AUD | USD |
| **USDCAD** | Loonie | USD | CAD |
| **USDCHF** | Swissie | USD | CHF |
| **NZDUSD** | Kiwi | NZD | USD |

### Base Currency vs Quote Currency

Dans une paire **EURUSD** :
- **Base currency** : EUR (premiere devise)
- **Quote currency** : USD (seconde devise)

**Prix EURUSD = 1.0850** signifie :
- 1 EUR = 1.0850 USD
- Pour acheter 1 EUR, il faut 1.0850 USD

**Actions** :
- **Acheter EURUSD (Long)** = Acheter EUR, Vendre USD
- **Vendre EURUSD (Short)** = Vendre EUR, Acheter USD

### Pip et Lot Size

**Pip** (Percentage in Point) : Plus petite variation de prix
- Pour la plupart des paires : 0.0001 (4e decimale)
- Pour JPY pairs : 0.01 (2e decimale)

**Lot Size** :
| Type | Unites | Valeur 1 pip (EURUSD) |
|------|--------|------------------------|
| **Standard Lot** | 100,000 | $10 |
| **Mini Lot** | 10,000 | $1 |
| **Micro Lot** | 1,000 | $0.10 |
| **Nano Lot** | 100 | $0.01 |

### Leverage en Forex

Le leverage permet de controler une grande position avec peu de capital :

| Leverage | Marge requise | Position controlable avec $1,000 |
|----------|---------------|-----------------------------------|
| 1:1 | 100% | $1,000 |
| 50:1 | 2% | $50,000 |
| 100:1 | 1% | $100,000 |
| 500:1 | 0.2% | $500,000 |

> **Attention** : Le leverage amplifie aussi les pertes ! Un mouvement de 1% contre vous avec leverage 100:1 = perte de 100% du capital.

In [ ]:
# Demonstration calcul pip et lot size

# EURUSD example
eurusd_price = 1.0850
pip_size = 0.0001

print("=== Calculs Forex ===")
print(f"\nPaire: EURUSD")
print(f"Prix: {eurusd_price}")
print(f"Pip size: {pip_size}")

# Valeur d'un pip pour different lot sizes
lot_sizes = {
    "Standard Lot": 100000,
    "Mini Lot": 10000,
    "Micro Lot": 1000,
}

print(f"\nValeur de 1 pip:")
for lot_name, lot_units in lot_sizes.items():
    pip_value = lot_units * pip_size
    print(f"  {lot_name} ({lot_units:,} units): ${pip_value:.2f}")

# Calcul P&L exemple
print(f"\n--- Exemple Trade ---")
entry_price = 1.0850
exit_price = 1.0870
pips_gained = (exit_price - entry_price) / pip_size
position_size = 10000  # Mini lot
pip_value = position_size * pip_size
pnl = pips_gained * pip_value

print(f"Entry: {entry_price}")
print(f"Exit: {exit_price}")
print(f"Pips gained: {pips_gained:.0f}")
print(f"Position: {position_size:,} units (mini lot)")
print(f"P&L: ${pnl:.2f}")

---

## 6. Ajouter des Forex dans QuantConnect (10 min)

### Brokers Forex Disponibles

QuantConnect supporte deux brokers Forex principaux :

| Broker | Description | Free Tier Data |
|--------|-------------|----------------|
| **Oanda** | Broker retail populaire | Oui (limite) |
| **FXCM** | Broker institutionnel | Oui (limite) |

### Syntaxe AddForex

In [ ]:
# Code QuantConnect - Ajouter des paires Forex
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class ForexBasicExample(QCAlgorithm):
    """
    Exemple d'ajout de paires Forex.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Methode 1: Oanda (recommande pour free tier)
        self.eurusd = self.AddForex(
            "EURUSD",
            Resolution.Hour,
            Market.Oanda
        )
        
        # Methode 2: FXCM
        self.gbpusd = self.AddForex(
            "GBPUSD",
            Resolution.Hour,
            Market.FXCM
        )
        
        # Ajouter plusieurs paires
        self.usdjpy = self.AddForex("USDJPY", Resolution.Hour, Market.Oanda)
        self.audusd = self.AddForex("AUDUSD", Resolution.Hour, Market.Oanda)
        
        self.Debug("Forex pairs added:")
        self.Debug(f"  EURUSD: {self.eurusd.Symbol}")
        self.Debug(f"  GBPUSD: {self.gbpusd.Symbol}")
        self.Debug(f"  USDJPY: {self.usdjpy.Symbol}")
        self.Debug(f"  AUDUSD: {self.audusd.Symbol}")
    
    def OnData(self, data):
        # Afficher les prix
        if data.ContainsKey(self.eurusd.Symbol):
            bar = data[self.eurusd.Symbol]
            self.Debug(f"{self.Time}: EURUSD Close={bar.Close:.5f}")

### Resolutions Disponibles

| Resolution | Description | Usage |
|------------|-------------|-------|
| `Resolution.Tick` | Chaque tick | Scalping, HFT |
| `Resolution.Second` | Chaque seconde | Intraday rapide |
| `Resolution.Minute` | Chaque minute | Intraday standard |
| `Resolution.Hour` | Chaque heure | Swing trading |
| `Resolution.Daily` | Chaque jour | Position trading |

### Principales Paires Supportees

**Majors** (USD d'un cote) :
```
EURUSD, GBPUSD, USDJPY, USDCHF, AUDUSD, USDCAD, NZDUSD
```

**Crosses** (sans USD) :
```
EURGBP, EURJPY, GBPJPY, AUDJPY, EURAUD, GBPAUD, CHFJPY
```

**Exotiques** (avec devises emergentes) :
```
USDMXN, USDZAR, USDTRY, EURTRY
```

---

## 7. Trading Forex (15 min)

### Positions Longues et Courtes

En Forex, vous pouvez facilement prendre des positions **short** (contrairement aux actions ou le short selling est plus complexe).

- **Long EURUSD** : Vous pensez que l'EUR va s'apprecier vs USD
- **Short EURUSD** : Vous pensez que l'EUR va se deprecier vs USD

In [ ]:
# Code QuantConnect - Trading Forex Long/Short
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class ForexTradingExample(QCAlgorithm):
    """
    Exemple de trading Forex avec positions longues et courtes.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter EURUSD
        self.eurusd = self.AddForex("EURUSD", Resolution.Hour, Market.Oanda)
    
    def OnData(self, data):
        if not data.ContainsKey(self.eurusd.Symbol):
            return
        
        # Exemple: Trading basique
        # (En pratique, utilisez des indicateurs et une vraie logique)
        
        # Long EURUSD (acheter EUR, vendre USD)
        # 10,000 units = 0.1 lot (mini lot)
        # self.MarketOrder("EURUSD", 10000)
        
        # Short EURUSD (vendre EUR, acheter USD)
        # Quantite negative = short
        # self.MarketOrder("EURUSD", -10000)
        
        # Ou avec SetHoldings (percentage du portfolio)
        # Long 50% du portfolio
        # self.SetHoldings("EURUSD", 0.5)
        
        # Short 50% du portfolio
        # self.SetHoldings("EURUSD", -0.5)
        
        pass

### Calcul des Lots

In [ ]:
# Code QuantConnect - Calcul des lots Forex
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class ForexLotCalculation(QCAlgorithm):
    """
    Exemple de calcul de la taille de position en Forex.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.eurusd = self.AddForex("EURUSD", Resolution.Hour, Market.Oanda)
        
        # Parametres de risk management
        self.risk_per_trade = 0.01  # 1% du capital par trade
        self.stop_loss_pips = 50     # Stop loss a 50 pips
    
    def CalculateLotSize(self, symbol, stop_loss_pips):
        """
        Calcule la taille de lot basee sur le risque.
        
        Args:
            symbol: Le symbole Forex
            stop_loss_pips: Nombre de pips jusqu'au stop loss
        
        Returns:
            int: Nombre d'unites a trader
        """
        # Capital disponible
        account_value = self.Portfolio.TotalPortfolioValue
        
        # Montant a risquer
        risk_amount = account_value * self.risk_per_trade
        
        # Valeur d'un pip (approximation)
        # Pour EURUSD, 1 pip = 0.0001, donc pour 1 unit: $0.0001
        pip_value_per_unit = 0.0001  # Pour paires XXX/USD
        
        # Calcul: risk_amount = lot_size * pip_value_per_unit * stop_loss_pips
        # lot_size = risk_amount / (pip_value_per_unit * stop_loss_pips)
        lot_size = risk_amount / (pip_value_per_unit * stop_loss_pips)
        
        # Arrondir aux 1000 (micro lot)
        lot_size = int(lot_size / 1000) * 1000
        
        return lot_size
    
    def OnData(self, data):
        if not data.ContainsKey(self.eurusd.Symbol):
            return
        
        if not self.Portfolio.Invested:
            # Calculer la taille de position
            lot_size = self.CalculateLotSize(self.eurusd.Symbol, self.stop_loss_pips)
            
            self.Debug(f"Lot size calcule: {lot_size:,} units")
            self.Debug(f"  = {lot_size / 100000:.2f} standard lots")
            self.Debug(f"  = {lot_size / 10000:.1f} mini lots")
            
            # Placer l'ordre
            # self.MarketOrder(self.eurusd.Symbol, lot_size)

### Spread et Slippage

En Forex, le **spread** (difference bid/ask) est le principal cout de transaction.

| Paire | Spread typique (pips) | Spread $ (mini lot) |
|-------|----------------------|--------------------|
| EURUSD | 0.5 - 1.5 | $0.50 - $1.50 |
| GBPUSD | 1.0 - 2.0 | $1.00 - $2.00 |
| USDJPY | 0.5 - 1.5 | $0.50 - $1.50 |
| Exotiques | 5 - 20 | $5 - $20 |

**Slippage** : Difference entre prix attendu et prix d'execution (plus important en volatilite elevee ou faible liquidite).

---

# PARTIE 3 : LEVERAGE ET RISK MANAGEMENT (20 min)

---

## 8. Gestion du Leverage (10 min)

### Leverage en Futures vs Forex

| Instrument | Leverage typique | Caracteristiques |
|------------|------------------|------------------|
| **Futures** | 10:1 - 20:1 | Marge fixe par contrat |
| **Forex (US)** | 50:1 max | Reglementation stricte |
| **Forex (EU)** | 30:1 max (majors) | ESMA rules depuis 2018 |
| **Forex (offshore)** | 100:1 - 500:1 | Non recommande |

### Acceder au BuyingPowerModel

In [ ]:
# Code QuantConnect - Verifier le leverage disponible
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class LeverageInfoExample(QCAlgorithm):
    """
    Affiche les informations de leverage pour differents instruments.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 1, 31)
        self.SetCash(100000)
        
        # Ajouter Forex
        self.eurusd = self.AddForex("EURUSD", Resolution.Hour, Market.Oanda)
        
        # Ajouter Future
        self.es = self.AddFuture(Futures.Indices.SP500EMini, Resolution.Hour)
        self.es.SetFilter(timedelta(0), timedelta(90))
        
        # Configurer leverage custom (optionnel)
        # self.eurusd.SetLeverage(50)  # 50:1
    
    def OnSecuritiesChanged(self, changes):
        for security in changes.AddedSecurities:
            self.Debug(f"\nSecurity: {security.Symbol}")
            self.Debug(f"  Type: {security.Type}")
            
            # Leverage info
            leverage = security.BuyingPowerModel.GetLeverage(security)
            self.Debug(f"  Leverage: {leverage}:1")
            
            # Margin requirements (pour Futures)
            if security.Type == SecurityType.Future:
                self.Debug(f"  Note: Futures use margin model, not simple leverage")

### Configurer le Leverage

Vous pouvez ajuster le leverage selon votre strategie :

```python
# Reduire le leverage (plus conservateur)
self.eurusd.SetLeverage(10)  # 10:1 au lieu de 50:1

# Ou via BuyingPowerModel
self.Securities["EURUSD"].SetBuyingPowerModel(
    SecurityMarginModel(leverage=10)
)
```

> **Recommandation** : Meme si un leverage eleve est disponible, utilisez un leverage effectif de **5:1 a 10:1 maximum** pour une gestion de risque raisonnable.

---

## 9. Position Sizing avec Leverage (10 min)

### Regle d'Or : Ne JAMAIS utiliser 100% du leverage

Utiliser le leverage maximum disponible est la principale cause de pertes catastrophiques en trading.

**Regles de position sizing** :
1. **Risquer max 1-2% du capital par trade**
2. **Utiliser max 20-30% du leverage disponible**
3. **Avoir des stop-loss systematiques**

In [ ]:
# Code QuantConnect - Position Sizing pour Futures
# A copier dans QuantConnect Lab

from AlgorithmImports import *

class FuturesPositionSizing(QCAlgorithm):
    """
    Strategie avec position sizing basee sur le risque.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # ES E-mini S&P 500
        self.es = self.AddFuture(Futures.Indices.SP500EMini, Resolution.Hour)
        self.es.SetFilter(timedelta(0), timedelta(90))
        
        # Parametres
        self.es_multiplier = 50  # $50 par point pour ES
        self.risk_per_trade = 0.01  # 1% du capital
        self.current_contract = None
    
    def CalculateFuturesQuantity(self, symbol, stop_loss_points):
        """
        Calcule le nombre de contrats Futures basee sur le risque.
        
        Args:
            symbol: Symbole du Future
            stop_loss_points: Distance du stop loss en points
        
        Returns:
            int: Nombre de contrats a trader
        """
        # Capital total
        account_value = self.Portfolio.TotalPortfolioValue
        
        # Montant a risquer
        risk_amount = account_value * self.risk_per_trade
        
        # Risque par contrat = stop_loss_points * multiplier
        risk_per_contract = stop_loss_points * self.es_multiplier
        
        # Nombre de contrats (arrondi vers le bas)
        num_contracts = int(risk_amount / risk_per_contract)
        
        return max(1, num_contracts)  # Minimum 1 contrat
    
    def OnData(self, slice):
        for chain in slice.FutureChains.Values:
            contracts = list(chain)
            if len(contracts) == 0:
                continue
            
            # Front month
            front = min(contracts, key=lambda x: x.Expiry)
            self.current_contract = front.Symbol
            
            if not self.Portfolio.Invested:
                # Exemple: stop loss a 20 points
                stop_loss_points = 20
                qty = self.CalculateFuturesQuantity(self.current_contract, stop_loss_points)
                
                self.Debug(f"Position Sizing:")
                self.Debug(f"  Account: ${self.Portfolio.TotalPortfolioValue:,.2f}")
                self.Debug(f"  Risk per trade: {self.risk_per_trade * 100}%")
                self.Debug(f"  Stop loss: {stop_loss_points} points")
                self.Debug(f"  Risk amount: ${self.Portfolio.TotalPortfolioValue * self.risk_per_trade:,.2f}")
                self.Debug(f"  Calculated quantity: {qty} contracts")
                
                # Entry example
                # self.MarketOrder(self.current_contract, qty)

In [ ]:
# Demonstration du calcul de position sizing

def calculate_futures_position(account_value, risk_pct, stop_loss_points, multiplier):
    """
    Calcule la taille de position pour Futures.
    """
    risk_amount = account_value * risk_pct
    risk_per_contract = stop_loss_points * multiplier
    num_contracts = int(risk_amount / risk_per_contract)
    return max(1, num_contracts)

# Parametres
account_value = 100000
risk_pct = 0.01  # 1%
es_multiplier = 50

print("=== Position Sizing Futures (ES) ===")
print(f"Account Value: ${account_value:,}")
print(f"Risk per Trade: {risk_pct * 100}%")
print(f"Risk Amount: ${account_value * risk_pct:,}")
print(f"ES Multiplier: ${es_multiplier}")
print()

# Test differents stop loss
for stop_loss in [10, 20, 50, 100]:
    qty = calculate_futures_position(account_value, risk_pct, stop_loss, es_multiplier)
    contract_value = 5000 * es_multiplier  # Assume ES at 5000
    total_exposure = qty * contract_value
    effective_leverage = total_exposure / account_value
    
    print(f"Stop Loss {stop_loss} pts: {qty} contracts (leverage {effective_leverage:.1f}x)")

### Tableau de Position Sizing

Pour un compte de $100,000 avec 1% de risque par trade :

| Stop Loss (ES) | Contrats | Exposure | Leverage Effectif |
|----------------|----------|----------|-------------------|
| 10 points | 20 | $5M | 50x |
| 20 points | 10 | $2.5M | 25x |
| 50 points | 4 | $1M | 10x |
| 100 points | 2 | $500k | 5x |

> **Recommandation** : Viser un leverage effectif de 5x a 10x maximum.

---

# PARTIE 4 : STRATEGIE COMPLETE (20 min)

---

## 10. Strategie Trend Following sur Futures (20 min)

### Specification de la Strategie

Nous allons implementer une strategie complete de **Trend Following** sur ES (E-mini S&P 500) :

**Instrument** : ES (E-mini S&P 500)

**Signal** : Breakout 20-day high/low (Donchian Channel)
- **Entry Long** : Prix casse le plus haut des 20 derniers jours
- **Entry Short** : Prix casse le plus bas des 20 derniers jours
- **Exit** : Casse du canal oppose sur 10 jours

**Position Sizing** : 1% de risque par trade

**Rollover** : Automatique via Continuous Contract

### Code Complet

In [ ]:
# Code QuantConnect - Strategie Complete Trend Following ES
# A copier dans QuantConnect Lab

from AlgorithmImports import *
from collections import deque

class ESTrendFollowingStrategy(QCAlgorithm):
    """
    Strategie Trend Following sur ES E-mini S&P 500.
    
    Signal: Donchian Channel Breakout (20/10)
    - Entry: Breakout 20-day high/low
    - Exit: Breakout 10-day opposite channel
    
    Position Sizing: 1% risk per trade
    Rollover: Automatic via Continuous Contract
    """
    
    def Initialize(self):
        # Configuration
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter ES avec Continuous Contract
        self.es = self.AddFuture(
            Futures.Indices.SP500EMini,
            Resolution.Daily
        )
        self.es.SetFilter(timedelta(0), timedelta(90))
        
        # Parametres de la strategie
        self.entry_period = 20   # Donchian entry period
        self.exit_period = 10    # Donchian exit period
        self.es_multiplier = 50  # $50 per point
        self.risk_per_trade = 0.01  # 1% risk
        
        # Tracking
        self.current_contract = None
        self.highs = deque(maxlen=self.entry_period)
        self.lows = deque(maxlen=self.entry_period)
        self.position_direction = 0  # 0: flat, 1: long, -1: short
        
        # Scheduling: Execute only once per day at market open
        self.Schedule.On(
            self.DateRules.EveryDay(self.es.Symbol),
            self.TimeRules.AfterMarketOpen(self.es.Symbol, 5),
            self.DailyCheck
        )
    
    def DailyCheck(self):
        """Daily logic check scheduled at market open."""
        pass  # Logic is in OnData for Futures
    
    def OnData(self, slice):
        """Main trading logic."""
        for chain in slice.FutureChains.Values:
            contracts = list(chain)
            if len(contracts) == 0:
                continue
            
            # Select front-month contract
            front = min(contracts, key=lambda x: x.Expiry)
            
            # Handle rollover
            if self.current_contract is not None and self.current_contract != front.Symbol:
                self._handle_rollover(front.Symbol)
            
            self.current_contract = front.Symbol
            
            # Get price data
            bar = slice.Bars.get(self.current_contract)
            if bar is None:
                continue
            
            # Update price history
            self.highs.append(bar.High)
            self.lows.append(bar.Low)
            
            # Wait for enough data
            if len(self.highs) < self.entry_period:
                return
            
            # Calculate Donchian Channels
            entry_high = max(list(self.highs)[-self.entry_period:])
            entry_low = min(list(self.lows)[-self.entry_period:])
            exit_high = max(list(self.highs)[-self.exit_period:])
            exit_low = min(list(self.lows)[-self.exit_period:])
            
            current_price = bar.Close
            
            # Trading Logic
            if self.position_direction == 0:  # Flat
                # Check for entry signals
                if current_price > entry_high:
                    self._enter_long(current_price, entry_low)
                elif current_price < entry_low:
                    self._enter_short(current_price, entry_high)
            
            elif self.position_direction == 1:  # Long
                # Check for exit signal (break below 10-day low)
                if current_price < exit_low:
                    self._exit_position("Break below 10-day low")
            
            elif self.position_direction == -1:  # Short
                # Check for exit signal (break above 10-day high)
                if current_price > exit_high:
                    self._exit_position("Break above 10-day high")
    
    def _enter_long(self, price, stop_loss_level):
        """Enter long position."""
        stop_loss_points = price - stop_loss_level
        qty = self._calculate_quantity(stop_loss_points)
        
        self.Debug(f"{self.Time}: ENTRY LONG")
        self.Debug(f"  Price: {price:.2f}")
        self.Debug(f"  Stop Loss: {stop_loss_level:.2f} ({stop_loss_points:.2f} points)")
        self.Debug(f"  Quantity: {qty} contracts")
        
        self.MarketOrder(self.current_contract, qty)
        self.position_direction = 1
    
    def _enter_short(self, price, stop_loss_level):
        """Enter short position."""
        stop_loss_points = stop_loss_level - price
        qty = self._calculate_quantity(stop_loss_points)
        
        self.Debug(f"{self.Time}: ENTRY SHORT")
        self.Debug(f"  Price: {price:.2f}")
        self.Debug(f"  Stop Loss: {stop_loss_level:.2f} ({stop_loss_points:.2f} points)")
        self.Debug(f"  Quantity: {qty} contracts")
        
        self.MarketOrder(self.current_contract, -qty)  # Negative for short
        self.position_direction = -1
    
    def _exit_position(self, reason):
        """Exit current position."""
        self.Debug(f"{self.Time}: EXIT - {reason}")
        self.Liquidate(self.current_contract)
        self.position_direction = 0
    
    def _calculate_quantity(self, stop_loss_points):
        """
        Calculate position size based on risk.
        
        Args:
            stop_loss_points: Distance to stop loss in points
        
        Returns:
            int: Number of contracts
        """
        account_value = self.Portfolio.TotalPortfolioValue
        risk_amount = account_value * self.risk_per_trade
        risk_per_contract = stop_loss_points * self.es_multiplier
        
        if risk_per_contract <= 0:
            return 1
        
        qty = int(risk_amount / risk_per_contract)
        return max(1, qty)
    
    def _handle_rollover(self, new_contract):
        """Handle contract rollover."""
        if self.Portfolio[self.current_contract].Invested:
            qty = self.Portfolio[self.current_contract].Quantity
            self.Debug(f"{self.Time}: ROLLOVER")
            self.Debug(f"  From: {self.current_contract}")
            self.Debug(f"  To: {new_contract}")
            self.Debug(f"  Quantity: {qty}")
            
            self.Liquidate(self.current_contract)
            self.MarketOrder(new_contract, qty)
    
    def OnEndOfAlgorithm(self):
        """Final summary."""
        self.Debug("="*50)
        self.Debug("STRATEGY SUMMARY")
        self.Debug("="*50)
        self.Debug(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")
        self.Debug(f"Initial Capital: $100,000.00")
        total_return = (self.Portfolio.TotalPortfolioValue - 100000) / 100000 * 100
        self.Debug(f"Total Return: {total_return:.2f}%")

### Explication de la Strategie

**1. Donchian Channel Breakout**

Le Donchian Channel est un indicateur de trend following cree par Richard Donchian :
- **Upper Band** : Plus haut des N derniers jours
- **Lower Band** : Plus bas des N derniers jours
- **Entry** : Prix casse le channel (20 jours)
- **Exit** : Prix casse le channel oppose (10 jours)

**2. Position Sizing**

La taille de position est calculee pour risquer exactement 1% du capital :
```
Quantity = (Account * Risk%) / (Stop Loss Points * Multiplier)
```

**3. Rollover Automatique**

Quand un nouveau contrat front-month est detecte, la position est transferee automatiquement.

### Points d'Amelioration Potentiels

Cette strategie de base peut etre amelioree avec :

1. **ATR-based stops** : Stop loss dynamique base sur l'Average True Range
2. **Trailing stops** : Stop suiveur pour securiser les profits
3. **Pyramiding** : Ajouter a la position sur continuation de tendance
4. **Multi-timeframe** : Confirmation sur timeframe superieur
5. **Risk-off filter** : Eviter de trader pendant haute volatilite (ex: VIX > 30)

---

## Conclusion

### Recapitulatif

Dans ce notebook, vous avez appris :

**Futures** :
- Structure des contrats Futures (marge, multiplicateur, expiration)
- `AddFuture()` avec Continuous Contract et filtre d'expiration
- FutureChain et selection de contrats (front-month, open interest)
- Gestion du rollover avec `OnSecuritiesChanged`

**Forex** :
- Paires de devises (base/quote currency)
- Pip, lot size, et calcul des profits
- `AddForex()` avec Oanda ou FXCM
- Positions longues et courtes

**Leverage et Risk** :
- Leverage effectif vs leverage maximum
- Position sizing avec 1% de risque
- Calcul de la quantite basee sur stop loss

**Strategie Complete** :
- Trend Following avec Donchian Channel
- Integration du rollover automatique
- Logs et debugging

### Points Cles a Retenir

| Concept | Recommendation |
|---------|----------------|
| **Futures** | Utiliser Continuous Contract + SetFilter |
| **Rollover** | Rouler 5-10 jours avant expiration |
| **Forex** | Utiliser Oanda (meilleur support free tier) |
| **Leverage** | Max 5-10x effectif |
| **Position Sizing** | 1-2% risque max par trade |
| **Stop Loss** | Toujours utiliser des stops |

### Prochaines Etapes

Dans les prochains notebooks, vous apprendrez :
- **QC-Py-08** : Options trading (calls, puts, strategies)
- **QC-Py-09** : Crypto trading
- **QC-Py-10** : Multi-asset portfolio strategies

### Exercice Suggere

**Implementer une strategie Mean Reversion sur Forex** :
1. Paire : EURUSD
2. Signal : RSI < 30 (long) / RSI > 70 (short)
3. Position sizing : 1% risk
4. Stop loss : 50 pips
5. Take profit : 100 pips

### Ressources

- [QuantConnect Futures Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/securities/asset-classes/futures)
- [QuantConnect Forex Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/securities/asset-classes/forex)
- [Turtle Trading Rules](https://bigpicture.typepad.com/comments/files/turtlerules.pdf) (original Donchian strategy)

---

**Notebook complete. Pret pour QC-Py-08-Options-Trading.**